# Miscellaneous Utilities
The follow demonstrates some miscellaneous utilities included in ObsPlus.

## Geodetics
Often it is necessary to calculate geometric parameters (distance, azimuth, etc.) for pairs of entities in two different groups. For example, distance from each event in a catalog to each receiver in an inventory. ObsPlus provides a simple class for doing just that.

In [1]:
# Load the catalog and inventory from the crandall dataset
import obsplus
from obsplus.utils.geodetics import SpatialCalculator


crandall = obsplus.load_dataset('crandall')
cat = crandall.event_client.get_events()
inv = crandall.station_client.get_stations()

In [2]:
# init a SpatialCalculator instance (defaults to Earth's params)
spatial_calc = SpatialCalculator()

In [3]:
# create distance dataframe
df = spatial_calc(entity_1=cat, entity_2=inv)
df.head()

,,distance_m,azimuth,back_azimuth,distance_degrees,vertical_distance_m
id1,id2,,,,,
smi:local/248887,TA.P18A..BHE,85335.022856,257.882101,77.264822,0.768112,6903.0
smi:local/248882,TA.P18A..BHE,86144.445871,258.177769,77.553917,0.775398,4533.0
smi:local/248883,TA.P18A..BHE,85972.920366,258.031261,77.409011,0.773854,6923.0
smi:local/248925,TA.P18A..BHE,85733.122444,257.957443,77.337106,0.771695,7363.0
smi:local/248891,TA.P18A..BHE,86139.480174,257.851636,77.228639,0.775353,5983.0


Since a multi-index is used it provides a fairly intuitive way to look up particular event-channel pairs using a tuple of (event_id, seed_id) in conjunction with the `.loc` DataFrame attribute like so:

In [4]:
event_id = str(cat[0].resource_id)
seed_id = 'UU.MPU..HHZ'

print(df.loc[(event_id, seed_id)])

distance_m             71094.602961
azimuth                  149.592196
back_azimuth             329.859507
distance_degrees           0.639925
vertical_distance_m     6069.000000
Name: (smi:local/248887, UU.MPU..HHZ), dtype: float64


In [5]:
# or just get a particular parameter
print(df.loc[(event_id, seed_id), 'azimuth'])

149.59219578


Of course the distances can be converted to km and we can describe the distributions:

In [6]:
# Convert add km columns and delete m columns
m_columns = [x for x in df.columns if x.endswith('m')]
km_columns = [x.replace('_m', '_km') for x in m_columns]

df_km = (
    df.assign(**{x: df[y] / 1000. for x,y in zip(km_columns, m_columns)}).
    drop(columns=m_columns)
)

# Calculate stats for source reseiver distances
df_km.describe().round(decimals=2)

,azimuth,back_azimuth,distance_degrees,distance_km,vertical_distance_km
count,408.00,408.00,408.00,408.00,408.00
mean,201.06,211.69,0.82,91.24,5.22
std,104.37,98.06,0.33,36.55,1.85
min,9.16,48.81,0.16,18.26,1.70
25%,126.47,122.45,0.63,70.46,3.69
50%,163.87,189.79,0.82,90.85,5.50
75%,293.44,307.27,1.12,124.36,6.25
max,358.12,344.77,1.30,144.00,9.31


## Time

Working with dates/times can be frustrating, especially since (as of 2020) ObsPy, numpy, and pandas all use slightly different methods for working with time. ObsPlus provides some utilities to make things a little easier.

In [7]:
from contextlib import suppress

import numpy as np
import obspy
import obsplus
from obsplus.utils import to_datetime64, to_timedelta64, to_utc

Note that all ObsPlus datafames use numpy/pandas datatypes. 

In [8]:
df = obsplus.events_to_df(obspy.read_events())

In [9]:
df['time']

0   2012-04-04 14:21:42.300
1   2012-04-04 14:18:37.000
2   2012-04-04 14:08:46.000
Name: time, dtype: datetime64[ns]

### Converting between time datatypes
We can convert one or all of these to ObsPy `UTCDateTime` objects when needed

In [10]:
to_utc(df['time'])

array([UTCDateTime(2012, 4, 4, 14, 21, 42, 300000),
       UTCDateTime(2012, 4, 4, 14, 18, 37),
       UTCDateTime(2012, 4, 4, 14, 8, 46)], dtype=object)

### Operations on time
Unlike with obspy's `UTCDateTime`, numpy requires using timedeltas to specify offsets.

In [11]:
# This raises a TypeError We need to specify units of offset with timedelta64
with suppress(TypeError):
    df['time'] + 1

In [12]:
df['time'] + np.timedelta64(3600, 's')

0   2012-04-04 15:21:42.300
1   2012-04-04 15:18:37.000
2   2012-04-04 15:08:46.000
Name: time, dtype: datetime64[ns]

In [13]:
# However, obspy assumes offsets to be in seconds
obspy.UTCDateTime('2020-01-03') + 3600

2020-01-03T01:00:00.000000Z

## Catalog Navigation

Navigating ObsPy's [QuakeML-based](https://quake.ethz.ch/quakeml/) `Catalog` can be difficult. 
